In [1]:
import numpy as np
import pandas as pd
import nibabel as nib
import random
import sys
import os
import scipy
import os.path as osp

sys.path.append('..')

from functions.visualization import polarAngle_plot, signMap_plot
from ipywidgets import interact, Dropdown

In [3]:
def transform(path):
    """
    Transform the polar angle maps from -180 to 180 degrees where the origin in the positive y-axis, to 0 to 360 degrees where
    the origin is the positive x-axis.
    
    Parameters
    ----------
    path : str
        The path to the freesurfer directory.
        
    Returns
    -------
    numpy.ndarray
        The transformed polar angle in degrees.
    """

    subjects = os.listdir(path)
    index_sub = subjects.index('processed') # to edit
    subjects.pop(index_sub)
    for hemisphere in ['lh', 'rh']:
        for subject in subjects:
            data = nib.load(path + '/'+ subject + '/surf/' + subject +
                                    '.empirical_polarAngle.' + hemisphere + '.32k_fs_LR.func.gii')
            angle = data.agg_data()
            
            # Step 1: Negate the angle
            angle = -angle
            # Step 2: Add 180 degrees
            angle = (angle + 180) % 360
            # Step 3: Rotate by 90 degrees using a rotation matrix
            x = np.cos(np.radians(angle))
            y = np.sin(np.radians(angle))
            rotation_matrix = np.array([[0, 1], [-1, 0]])
            rotated_coords = rotation_matrix @ np.array([x, y])
            rotated_angle = np.degrees(np.arctan2(rotated_coords[1], rotated_coords[0]))
            final_angle = rotated_angle % 360

            data.agg_data()[:] = final_angle
            nib.save(data, path + '/' + subject + '/surf/' + subject +
                                    '.empirical_polarAngle_test.' + hemisphere + '.32k_fs_LR.func.gii')
    return 

In [4]:
transform('../datasets/ds004698/derivatives/freesurfer')

In [2]:
path_to_freesurfer = '../datasets/ds004698/derivatives/freesurfer/'
subject_id= os.listdir(path_to_freesurfer)
index_sub = subject_id.index('processed') # to edit
subject_id.pop(index_sub)
prediction = Dropdown(options = ['average', 'empirical','model1', 'model2', 'model3', 'model4', 'model5'])
binarize = Dropdown(options = [False, True])

@interact(subject_id=subject_id, prediction = prediction, binarize = binarize)
def plot1(subject_id,prediction, binarize):
    return polarAngle_plot(subject_id, path_to_freesurfer, '../templates', prediction = prediction, binarize = binarize)

interactive(children=(Dropdown(description='subject_id', options=('sub-01', 'sub-02', 'sub-03', 'sub-04', 'sub…

In [6]:
@interact(subject_id=subject_id)
def plot2(subject_id):
    return signMap_plot(subject_id, path_to_freesurfer, '../templates', hemisphere = 'lh')


interactive(children=(Dropdown(description='subject_id', options=('sub-01', 'sub-02', 'sub-03', 'sub-04', 'sub…